In [1]:
%load_ext autoreload
%autoreload 2

from PIL import Image
import numpy as np
from matplotlib.pylab import plt

from mtcnn import detect_faces, show_bboxes
from solov2 import nuki_faces

### 1. Target Image

In [3]:
img = Image.open('samples/trot.png')
Image.fromarray(np.asarray(img)[:,:,:3]).save("samples/trot.jpg")

In [ ]:
# Get BBoxs
img = Image.open('samples/trot.jpg')
bboxs, landmarks = detect_faces(img)

# Convert to Nuki Boxs
nboxs = []
for bbox in bboxs:
    x0, y0, x1, y1 = bbox[:4]
    xc = (x0+x1)/2
    yc = (y0+y1)/2
    w = int((x1-x0)/2 * 2.2)
    h = (y1-y0)/2
    nboxs.append((int(xc-w), int(yc-h*2), int(xc+w), int(yc+h)))
    
masks = nuki_faces(img, nboxs)


[01/16 01:14:55 detectron2]: Arguments: {'config_file': 'solov2/configs/SOLOv2/R50_3x.yaml', 'input': ['jsm.jpg'], 'output': 'results', 'confidence_threshold': 0.5, 'opts': ['MODEL.WEIGHTS', 'solov2/weights/SOLOv2_R50_3x.pth']}


/Users/dajinhan/anaconda3/envs/face/lib/python3.8/site-packages/torch/nn/functional.py:3060: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn("Default upsampling behavior when mode={} is changed "
/Users/dajinhan/anaconda3/envs/face/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "
/Users/dajinhan/Workspace/AdelaiDet/adet/modeling/solov2/solov2.p

### 2. SizeUp Head

In [ ]:
orig_img = np.asarray(img)
result_img = np.asarray(img).copy()

zip_list = zip(nboxs, masks)
zip_list = sorted(zip_list, key=lambda x : x[1].sum())
for nbox, mask in zip_list:
    # original size
    x0,y0,x1,y1 = nbox[:4]
    xc = int((x0+x1)/2)
    yc = int((y0+y1)/2)
    w = x1-x0
    h = y1-y0
    
    # crop and resize
#     face_img = orig_img[y0:y1, x0:x1, :] * np.expand_dims(mask, 2)
#     face_img = Image.fromarray(face_img)
#     face_img = face_img.resize((int(w*1.2), int(h*1.2)))
#     face_img = np.asarray(face_img)
    face_img = orig_img[y0:y1, x0:x1, :]
    face_img = Image.fromarray(face_img)
    mask = Image.fromarray(mask.astype(np.uint8))
    
    face_img = face_img.resize((int(w*1.4), int(h*1.4)))
    mask = mask.resize((int(w*1.4), int(h*1.4)))
    
    face_img = np.asarray(face_img).copy()
    mask = np.asarray(mask)
    mask = np.expand_dims(mask, 2)
    face_img *= mask

    # new size
    nw = face_img.shape[1]
    nh = face_img.shape[0]
    for row in range(face_img.shape[0]):
        srow = row+yc+int(h/2-nh)
        if srow < 0 or srow >= result_img.shape[0]:
            continue
        for col in range(face_img.shape[1]):
            scol = col+xc-int(nw/2)
            if scol < 0 or scol >= result_img.shape[1]:
                continue
            
            if np.sum(face_img[row][col]) == 0:
                continue
                
            result_img[srow][scol] = face_img[row][col]
    
    
    

In [ ]:
plt.imshow(result_img)
Image.fromarray(result_img).save("results/result.png")